В клетках ниже - процесс обучения и запуска классификатора для существительных-субъектов

классификация существительных конечной точки устроена аналогичным образом, отличаясь тренировочным сетом и количеством классов

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
nouns = []

with open ('padat_nouns.txt', 'r', encoding = 'utf-8') as pf:
    c = pf.read()
    for ent in c.split('\n'):
        if int(ent.split('\t')[1]) >= 5:
            nouns.append(ent.split('\t')[0])

In [ ]:
nouns_to_vec = [n+'_NOUN' for n in nouns] #train nouns
nouns_to_class = [n+'_NOUN' for n in dataframe['nouns']] #nouns to classify

In [ ]:
h = list(set(nouns_to_vec))

In [ ]:
vecs = []
ws = []

for w in h:
    try:
        vec = wv_from_bin[w]
        vecs.append(vec)
        ws.append(w)
    except KeyError:
        pass

In [ ]:
X = np.array(vecs)

In [ ]:
kmeans = KMeans(n_clusters=n_classes).fit(X)

In [ ]:
classified = []

for word in nouns_to_class:
    if word == 'No subject':
        classified.append(None)
    else:
        try:
            vect = wv_from_bin[word].reshape(1, -1)
            classified.append(kmeans.predict(vect)[0])
        except KeyError:
            classified.append('Not in word2vec')

Создание векторов глагол+конечная точка

In [ ]:
gv_vec = []

for go, ve in zip(goal_list, verb_list):
    print(go, ve)
    try:
        g = wv_from_bin[go]
    except TypeError:
        g = 0
    v = wv_from_bin[ve+'_VERB']
    if type(g)!= np.array:
        gv_vec.append(v)
    else:
        gv_vec.append(np.add(g, v))

Разделение размеченной выборки на train и test для отладки

In [ ]:
subj_pos = np.vstack(train_df.subj_pos.values)
class_42 = np.vstack(train_df.subj_class_freq_42.values)
gv = np.vstack(train_df.goalVerb_vec.values)
gc = np.vstack(train_df.goal_class.values)


X = np.hstack((subj_pos, class_42,gv, gc))
y = train_df.frame.values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 10)

Random forest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = { 
    'n_estimators': [10, 50],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8, 9, 10],
    'criterion' :['gini', 'entropy'],
    'verbose':[1]
}

In [ ]:
def dtree_grid_search(X,y,nfolds):
    #create a dictionary of all values we want to test
    param_grid = { 
    'n_estimators': [30, 40, 50, 60, 70],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : np.arange(5, 15),
    'criterion' :['gini', 'entropy'],
    'verbose':[1]}
    tree_model=RandomForestClassifier()
    #use gridsearch to test all values
    dtree_gscv = GridSearchCV(tree_model, param_grid, cv=nfolds)
    #fit model to data
    dtree_gscv.fit(X, y)
    return dtree_gscv.best_params_

In [ ]:
dtree_grid_search(X, y, 10)

In [ ]:
rfc = RandomForestClassifier(criterion='gini', max_depth=9, max_features='sqrt', verbose=1, n_estimators= 40, min_samples_split=2)

In [ ]:
rfc.fit(X_train, y_train)

In [ ]:
y_pred = rfc.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=['f1', 'f2', 'f3', 'f4']))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
df_cm = pd.DataFrame(cm, range(4), range(4))

sn.set(font_scale=1.0) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 20}) # font size

plt.show()

Предсказания на всём корпусе

In [ ]:
subj_pos = np.vstack(ALL_CORP.subj_pos.values)
class_42 = np.vstack(ALL_CORP.subj_class_freq_42.values)
gv = np.vstack(ALL_CORP.goalVerb_vec.values)
gc = np.vstack(ALL_CORP.goal_class.values)


ac = np.hstack((subj_pos, class_42, gc, gv))

In [ ]:
predicted = rfc.predict(ac)